# Sharding for Write Scalability

In the last notebook, we saw that replication is excellent for scaling read requests, but it doesn't solve the **write bottleneck**—all `INSERT`, `UPDATE`, and `DELETE` operations must still go through a single Primary server.

When an application's write volume becomes too high for even the most powerful single server, we must turn to a more complex horizontal scaling technique: **Sharding**.

This notebook explains what sharding is, common strategies for implementing it, and the immense architectural challenges it presents.

--- 
## 1. What is Sharding?

**Sharding**, also known as **horizontal partitioning**, is the process of splitting a large database into many smaller, faster, more manageable parts called **shards**. Each shard is an independent database that runs on its own server and contains a unique subset of the total data.

Unlike replication (which copies all data to multiple servers), sharding splits the data up. No single server holds the complete dataset.

#### Analogy: The Gigantic Encyclopedia

You can't print a single book containing all human knowledge. Instead, you create an encyclopedia set. Volume A-C is one book (**Shard 1**), Volume D-F is another (**Shard 2**), and so on. To find information about "Dinosaurs", you don't search the whole library; you go directly to the "D-F" volume. A routing layer in front of the database acts as the encyclopedia's index, telling your application which shard to query.

### The Shard Key

The most critical decision in a sharded architecture is choosing the **shard key**. This is a column (or set of columns) in a table that the system uses to determine which shard a particular row belongs to. Common shard keys include `user_id`, `customer_id`, or `region`.

--- 
## 2. Common Sharding Strategies

How you use the shard key to distribute data determines the characteristics of your cluster.

### Range-Based Sharding
- **How it works**: Data is sharded based on a range of the shard key (e.g., User IDs 1-1,000,000 go to Shard A, 1,000,001-2,000,000 go to Shard B).
- **Pros**: Simple to implement and efficient for queries that scan ranges (e.g., "get all users who signed up in January").
- **Cons**: Can lead to **hotspots**. If one range is much more active (e.g., a new product launch causes a spike in new user IDs), that shard will be overloaded while others are idle.

### Hash-Based Sharding
- **How it works**: A hash function is applied to the shard key (e.g., `hash(user_id) % number_of_shards`). The result determines the shard.
- **Pros**: Distributes data very evenly across all shards, preventing hotspots.
- **Cons**: Makes range queries very difficult, as logically adjacent data (e.g., `user_id` 100 and 101) will likely be on different shards. To get a range, the application would have to query every single shard.

### Directory-Based Sharding
- **How it works**: A separate lookup table (a "directory") explicitly maps each shard key value to its shard location.
- **Pros**: Extremely flexible. You can move individual tenants or users between shards just by updating the lookup table.
- **Cons**: The lookup table itself can become a single point of failure and a performance bottleneck.

--- 
## 3. The Immense Challenges of Sharding

Sharding is incredibly powerful, but it introduces significant complexity. It is not a decision to be taken lightly.

#### Cross-Shard Joins
Performing a `JOIN` on data that lives on different shards is a distributed systems nightmare. It's extremely slow and complex because the database has to make network requests between shards to gather the data. This often forces teams to **denormalize** their data (i.e., store duplicate copies of information) to ensure that data that needs to be joined lives on the same shard.

#### Transactions and Consistency
Guaranteeing ACID transactions across multiple servers is very difficult. A simple `COMMIT` is no longer atomic. This requires complex protocols like **two-phase commits (2PC)**, which are slower and can leave the system in an inconsistent state if a server fails mid-transaction. Many sharded systems choose to relax their consistency guarantees for better performance and availability.

#### Rebalancing (Resharding)
What happens when you need to add a new shard to your cluster because you're growing? The process of moving potentially terabytes of data between live production servers without downtime is a massive and risky operational challenge.

#### Schema Changes
Applying a simple schema change (like adding a column) is no longer a single command. You now have to coordinate that change across dozens or hundreds of independent databases, which is a complex deployment task.

--- 
## Conclusion

Sharding is the ultimate solution for write scalability, allowing an application to handle a virtually infinite write load. However, it fundamentally changes the architecture from a simple, consistent database to a complex distributed system.

It is a measure of last resort, to be adopted only when the write load has outgrown the largest possible single primary server. Before sharding, it is almost always better to first exhaust other options, such as **caching**.